# FINAL MODEL

## *Data prep*

In [ ]:
df2 = pd.read_csv('data/KingSchool.csv')

In [ ]:
categoricals = ['District']
dummies = pd.get_dummies(df2[categoricals], prefix=categoricals, drop_first=True)
Kings_preprocessed = df2.drop(categoricals, axis=1)
Kings_preprocessed = pd.concat([Kings_preprocessed, dummies], axis=1)

In [ ]:
Kings_preprocessed.drop(['date'],axis=1,inplace=True)
Kings_preprocessed = Kings_preprocessed.astype(np.float64)
Kings_preprocessed['District_FEDERAL_WAY'] = Kings_preprocessed['District_FEDERAL WAY']
Kings_preprocessed['District_LAKE_WASHINGTON'] = Kings_preprocessed['District_LAKE WASHINGTON']
Kings_preprocessed['District_MERCER_ISLAND'] = Kings_preprocessed['District_MERCER ISLAND']
Kings_preprocessed['District_SNOQUALMIE_VALLEY'] = Kings_preprocessed['District_SNOQUALMIE VALLEY']
Kings_preprocessed['District_VASHON_ISLAND'] = Kings_preprocessed['District_VASHON ISLAND']
Kings_preprocessed.drop(['District_FEDERAL WAY','District_LAKE WASHINGTON','District_MERCER ISLAND',
                         'District_SNOQUALMIE VALLEY','District_VASHON ISLAND'],axis=1,inplace=True)

In [ ]:
King_District_Dummies = ['District_BELLEVUE','District_ENUMCLAW','District_FEDERAL_WAY','District_HIGHLINE',
                         'District_ISSAQUAH','District_KENT','District_LAKE_WASHINGTON','District_MERCER_ISLAND',
                         'District_NORTHSHORE','District_RENTON','District_RIVERVIEW','District_SEATTLE',
                         'District_SHORELINE','District_SKYKOMISH','District_SNOQUALMIE_VALLEY','District_TAHOMA',
                         'District_TUKWILA','District_VASHON_ISLAND']

In [ ]:
Kings_preprocessed.drop(['id','bedrooms','bathrooms','sqft_lot','floors','waterfront',
                         'view','condition','sqft_above','sqft_basement', 'yr_renovated',
                         'zipcode','lat','long','sqft_living15','sqft_lot15','renovated_less_10yrs',
                         'has_basement','Distance_to_School_ft','mean_price'], axis=1,inplace=True)

In [ ]:
final_y = Kings_preprocessed["price"]
final_X = Kings_preprocessed.drop("price", axis=1)
final_formula = ['sqft_living', 'grade', 'yr_built', 'District_BELLEVUE','District_ENUMCLAW','District_FEDERAL_WAY',
                 'District_HIGHLINE','District_ISSAQUAH','District_KENT','District_LAKE_WASHINGTON','District_MERCER_ISLAND',
                 'District_NORTHSHORE','District_RENTON','District_RIVERVIEW','District_SEATTLE','District_SHORELINE',
                 'District_SKYKOMISH','District_SNOQUALMIE_VALLEY','District_TAHOMA','District_TUKWILA',
                 'District_VASHON_ISLAND']
final_X = final_X[final_formula]

In [ ]:
final_X_train, final_X_test, final_y_train, final_y_test = train_test_split(final_X, final_y, random_state=42)

In [ ]:
scaler = StandardScaler()
final_X_train_scaled = scaler.fit_transform(final_X_train)
final_X_test_scaled = scaler.fit_transform(final_X_test)

In [ ]:
final_model = LinearRegression()
final_model.fit(final_X_train_scaled, final_y_train)
final_model.score(final_X_test_scaled, final_y_test)

In [ ]:
print(f'Train RMSE: {mean_squared_error(final_y_train, final_model.predict(final_X_train_scaled), squared=False)}')
print(f'Test RMSE: {mean_squared_error(final_y_test, final_model.predict(final_X_test_scaled), squared=False)}'

In [ ]:
sm.OLS(final_y_train, sm.add_constant(final_X_train)).fit().summary()

In [ ]:
final_X_train_scaled = pd.DataFrame(final_X_train_scaled)
final_X_train_scaled.columns = final_X.columns
final_X_test_scaled = pd.DataFrame(final_X_test_scaled)
final_X_test_scaled.columns = final_X.columns

# Assumptions

In [ ]:
# Linearity
linearity_plot(final_X_train, final_y_train)
#Dummy variables meet the assumption of linearity by definition

In [ ]:
#NORMALITY
final_X_test_scaled = pd.DataFrame(final_X_test_scaled)
preds_final = final_model.predict(final_X_test_scaled)
residuals = (final_y_test - preds_final)
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True);

In [ ]:
#Checking MULTICOLLINEARITY 
# (VIF NEEDS TO BE <5 ALL CATEGORIES)
vif = [variance_inflation_factor(final_X_train_scaled.values, i) for i in range(final_X_train_scaled.shape[1])]
pd.Series(vif, index=final_X_train_scaled.columns, name="Variance Inflation Factor")

In [ ]:

y_hat_train = final_model.predict(final_X_train_scaled)
y_hat_test = preds_final

sm.graphics.qqplot(final_y_train-y_hat_train,line='r')
plt.show()